# Description

TODO

# Environment variables

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import conf

In [3]:
N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

3

In [4]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

env: MKL_NUM_THREADS=3
env: OPEN_BLAS_NUM_THREADS=3
env: NUMEXPR_NUM_THREADS=3
env: OMP_NUM_THREADS=3


# Modules

In [5]:
from pathlib import Path

import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from tqdm import tqdm

from gls import GLSPhenoplier

# Settings

In [6]:
N_SIMULATED_PHENOTYPES = 10

In [7]:
OUTPUT_DIR = conf.RESULTS["GLS"]
display(OUTPUT_DIR)

OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

PosixPath('/opt/data/results/gls')

In [8]:
OUTPUT_FILENAME = OUTPUT_DIR / "gls-null_simulations.pkl"
display(OUTPUT_FILENAME)

PosixPath('/opt/data/results/gls/gls-null_simulations.pkl')

# Load data

## MultiPLIER Z matrix

In [9]:
multiplier_z_matrix = pd.read_pickle(conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"])

In [10]:
multiplier_z_matrix.shape

(6750, 987)

In [11]:
multiplier_z_matrix.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
GAS6,0.000000,0.0,0.039438,0.0,0.050476,0.000000,0.0,0.000000,0.590949,0.000000,...,0.050125,0.00000,0.033407,0.000000,0.000000,0.005963,0.347362,0.0,0.000000,0.000000
MMP14,0.000000,0.0,0.000000,0.0,0.070072,0.000000,0.0,0.004904,1.720179,2.423595,...,0.000000,0.00000,0.001007,0.000000,0.035747,0.000000,0.000000,0.0,0.014978,0.000000
DSP,0.000000,0.0,0.000000,0.0,0.000000,0.041697,0.0,0.005718,0.000000,0.000000,...,0.020853,0.00000,0.000000,0.000000,0.000000,0.005774,0.000000,0.0,0.000000,0.416405
MARCKSL1,0.305212,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.161843,0.149471,...,0.027134,0.05272,0.000000,0.030189,0.060884,0.000000,0.000000,0.0,0.000000,0.448480
SPARC,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.014014,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.067779,0.0,0.122417,0.062665


In [12]:
lv_codes = list(multiplier_z_matrix.columns)
display(lv_codes[:5])

['LV1', 'LV2', 'LV3', 'LV4', 'LV5']

## PhenomeXcan (S-MultiXcan)

In [13]:
# INPUT_SUBSET = "z_score_std"

In [14]:
# INPUT_STEM = "projection-smultixcan-efo_partial-mashr-zscores"

In [15]:
# input_filepath = Path(
#     conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
#     INPUT_SUBSET,
#     f"{INPUT_SUBSET}-{INPUT_STEM}.pkl",
# ).resolve()

In [16]:
# data = pd.read_pickle(input_filepath)

In [17]:
# data.shape

In [18]:
# data.head()

# GLSPhenoplier

## Load `lv_weights`

In [19]:
lv_weights = GLSPhenoplier._get_data(
    conf.PHENOMEXCAN["SMULTIXCAN_EFO_PARTIAL_MASHR_ZSCORES_FILE"]
)[2]

In [20]:
lv_weights.shape

(6450, 987)

In [21]:
lv_weights.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
GAS6,0.000000,0.0,0.039438,0.0,0.050476,0.000000,0.0,0.000000,0.590949,0.000000,...,0.050125,0.00000,0.033407,0.000000,0.000000,0.005963,0.347362,0.0,0.000000,0.000000
MMP14,0.000000,0.0,0.000000,0.0,0.070072,0.000000,0.0,0.004904,1.720179,2.423595,...,0.000000,0.00000,0.001007,0.000000,0.035747,0.000000,0.000000,0.0,0.014978,0.000000
DSP,0.000000,0.0,0.000000,0.0,0.000000,0.041697,0.0,0.005718,0.000000,0.000000,...,0.020853,0.00000,0.000000,0.000000,0.000000,0.005774,0.000000,0.0,0.000000,0.416405
MARCKSL1,0.305212,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.161843,0.149471,...,0.027134,0.05272,0.000000,0.030189,0.060884,0.000000,0.000000,0.0,0.000000,0.448480
SPARC,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.014014,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.067779,0.0,0.122417,0.062665


## Run

In [22]:
rs = np.random.RandomState(0)

In [23]:
results = []

pbar = tqdm(total=int(N_SIMULATED_PHENOTYPES * len(lv_codes)))

for idx in range(N_SIMULATED_PHENOTYPES):
    # generate a random phenotype
    phenotype_code = f"random_normal-{idx}"

    phenotype = pd.Series(
        # use abs to simulate MultiPLIER z-scores (always positives)
        np.abs(rs.normal(size=lv_weights.shape[0])),
        index=lv_weights.index.copy(),
        name=phenotype_code,
    )

    # compute an association for all LVs
    for lv_code in lv_codes[:3]:
        pbar.set_description(f"{phenotype_code} - {lv_code}")

        gls_model = GLSPhenoplier(
            smultixcan_result_set_filepath=conf.PHENOMEXCAN[
                "SMULTIXCAN_EFO_PARTIAL_MASHR_ZSCORES_FILE"
            ]
        )
        gls_model.fit_named(lv_code, phenotype)
        res = gls_model.results

        results.append(
            {
                "phenotype": phenotype_code,
                "lv": lv_code,
                "coef": res.params.loc["lv"],
                "pvalue": res.pvalues_onesided.loc["lv"],
                "pvalue_twosided": res.pvalues.loc["lv"],
                "summary": gls_model.results_summary,
            }
        )

        # save results every 10 models trained
        if (idx % 10) == 0:
            pd.DataFrame(results).to_pickle(OUTPUT_FILENAME)

        pbar.update(1)

pbar.close()

random_normal-9 - LV3:   0%|          | 30/9870 [04:36<25:13:51,  9.23s/it]


In [24]:
results = pd.DataFrame(results)

In [25]:
results.shape

(30, 6)

In [26]:
results.head()

,phenotype,lv,coef,pvalue,pvalue_twosided,summary
0,random_normal-0,LV1,0.010088,0.208194,0.416388,GLS Regression Res...
1,random_normal-0,LV2,0.003398,0.392886,0.785772,GLS Regression Res...
2,random_normal-0,LV3,0.008308,0.256323,0.512647,GLS Regression Res...
3,random_normal-1,LV1,0.013093,0.144843,0.289685,GLS Regression Res...
4,random_normal-1,LV2,0.004008,0.373797,0.747594,GLS Regression Res...


In [27]:
results.sort_values("pvalue").head(10)

,phenotype,lv,coef,pvalue,pvalue_twosided,summary
24,random_normal-8,LV1,0.021787,0.039785,0.079571,GLS Regression Res...
25,random_normal-8,LV2,0.017532,0.080626,0.161251,GLS Regression Res...
3,random_normal-1,LV1,0.013093,0.144843,0.289685,GLS Regression Res...
0,random_normal-0,LV1,0.010088,0.208194,0.416388,GLS Regression Res...
18,random_normal-6,LV1,0.008375,0.249024,0.498048,GLS Regression Res...
2,random_normal-0,LV3,0.008308,0.256323,0.512647,GLS Regression Res...
8,random_normal-2,LV3,0.006218,0.311497,0.622993,GLS Regression Res...
15,random_normal-5,LV1,0.005684,0.323406,0.646812,GLS Regression Res...
26,random_normal-8,LV3,0.005456,0.333808,0.667617,GLS Regression Res...
4,random_normal-1,LV2,0.004008,0.373797,0.747594,GLS Regression Res...


## Save

In [28]:
results.to_pickle(OUTPUT_FILENAME)